In [1]:
!pip3 install bs4
!pip install spacy

import nltk
import math
import os
import fnmatch
import random
import numpy as np
from xml.etree import ElementTree
import re
import string
import spacy
import scipy
from scipy import stats
from scipy.spatial import distance
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup as bs

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words

def preprocessText(text):

  def makeTextLower(text):
    result = text.lower() 
    return result 

  def MakeNumberRemove(text): 
    #result = re.sub(r'^\d+\s|\s\d+\s|\s\d+$', '', text)
    result = re.sub(r'[-+]?\d+', '', text) 
    return result 

  def MakeHTMLremove(text):
    cleancode = re.compile('<code>.*?</code>')
    cleanr = re.compile('<.*?>')
    cleanentity = re.compile('&.*;')
    cleantext = re.sub(cleancode, '', text)
    cleantext = re.sub(cleanr, ' ', cleantext)
    cleantext = re.sub(cleanentity, ' ', cleantext)  
    return cleantext

  def makeRemoveExtraApace(text):
    result = " ".join(text.split())  
    return result

  def makeRemoveHyperLink(text):
    result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return result  

  def makeRemovePunctuation(text):
    result = text=text.translate((str.maketrans('','',string.punctuation)))
    return result
  
  def makeWordLemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word,pos='v') for word in text] 
    return lemmas

  def removeStopWords(text):
    stopwords_english = stopwords.words('english')
    texts_clean = []
    #print(text)
    text = word_tokenize(text)
    for word in text: 
      if word not in stopwords_english:  
          texts_clean.append(word)
    return texts_clean

  def removeSpacyStopWords(text,spacy_stopwords):
    texts_clean = []
    #print(text)
    for word in text: 
      if word not in spacy_stopwords:  
          texts_clean.append(word)
    return texts_clean

  def makeStemming(text):
    ps = PorterStemmer() 
    #words = word_tokenize(text)
    lemmas = [ps.stem(word) for word in text] 
    return lemmas  

  result = text
  result=re.sub(r"<.+?>",'',result)
  result=re.sub(r'[^\x00-\x7F]+','',result)
  result = MakeHTMLremove(result)
  result = makeTextLower(result)
  result = MakeNumberRemove(result)
  result = makeRemoveHyperLink(result)
  result = makeRemovePunctuation(result)
  result = removeStopWords(result)
  result = removeSpacyStopWords(result,spacy_stopwords)
  result = makeWordLemmatize(result)
  result = makeStemming(result)
  return result  

def naiveBayes(allResult,trainDataSize,validationDataSize,topic_list,topicLength,smfactorList,type):
  trainDataSize = trainDataSize
  validationDataSize = validationDataSize
  def trainNaive(allResult,topicLength,smFactor):
  
    uniqueWord = []
    for word in allResult.keys():
      if word[0] not in uniqueWord:
        uniqueWord.append(word[0])
    uniqueWordSize = len(uniqueWord)
    #print(uniqueWord[:50])

    topicWiseCount = {}
    for wordPair in allResult.keys():
      if wordPair in topicWiseCount.keys():
        topicWiseCount[wordPair[1]] = topicWiseCount[wordPair[1]] + allResult[wordPair]
      else :
        topicWiseCount[wordPair[1]] = 1

    laplaceSmoothingCalc = {}
    for word in uniqueWord:
      for topic in range(topicLength):
        
        wordPair = (word,topic)
        prevCount = 0
        if wordPair in allResult:
          prevCount = allResult[wordPair]
        probability = (prevCount+smFactor)/(float(topicWiseCount[topic])+smFactor*uniqueWordSize)

        laplaceSmoothingCalc[wordPair] = probability
    
    return topicWiseCount,laplaceSmoothingCalc,uniqueWordSize

  def predictNaive(word_list,laplaceSmoothingCalc,topicLength,topicWiseCount,uniqueWordSize,smFactor):
  
    maxProb = 0.0
    currTopic = 0
    allDistances = []
    for topic in range(topicLength):
      multiProb = math.log(1/float(topicLength))
      for word in word_list:
        wordPair = (word,topic)
        if wordPair in laplaceSmoothingCalc.keys():
          multiProb = multiProb + math.log(laplaceSmoothingCalc[wordPair])
        else :
          multiProb = multiProb + math.log((smFactor)/(topicWiseCount[topic]+smFactor*uniqueWordSize))
          #continue
      #print(multiProb)
      allDistances.append((topic,word,multiProb))
    allDistances.sort(key=lambda x: x[2],reverse=True)
    currTopic = allDistances[0][0]
    #print()
    return currTopic
  #print(trainDataSize)
  if type is 'val':
    for smFactor in smfactorList:
      print("smoothing factor constant :",smFactor)
      topicWiseCount,laplaceSmoothingCalc,uniqueWordSize = trainNaive(allResult,topicLength,smFactor)
      #print(laplaceSmoothingCalc[("get",1)])
      #print(uniqueWordSize)
      correctCount = 0
      totalCount = 0
      for key in topic_list:
        path = "/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/Training/"+key+".xml"
        trainDataSize = trainDataSize
        validationDataSize = validationDataSize
        #print(trainDataSize)
        with open(path,'r',encoding='utf-8') as file:
          content = file.read()
          soup = bs(content)
          rowList = soup.findAll("row")
          #correctCount = 0
          #print(trainDataSize)
          for row in rowList[trainDataSize:trainDataSize+validationDataSize+1]:
            #print(row.get('body'))
            #print()
            if len(row.get('body')) == 0:
              continue
            word_list = preprocessText(row.get('body'))
            if len(word_list) == 0:
              continue
            if predictNaive(word_list,laplaceSmoothingCalc,topicLength,topicWiseCount,uniqueWordSize,smFactor) is topic_list[key]:
              correctCount += 1
            totalCount += 1
      print((correctCount/float(totalCount))*100)
      print()
    return correctCount/float(totalCount)

  elif type is 'test':
    for smFactor in smfactorList:
      print("smoothing factor constant :",smFactor)
      topicWiseCount,laplaceSmoothingCalc,uniqueWordSize = trainNaive(allResult,topicLength,smFactor)
      #print(laplaceSmoothingCalc[("get",1)])
      #print(uniqueWordSize)
      totalLoop = 3
      testSize = 10
      acc = []
      for loop in range(totalLoop):
        correctCount = 0
        totalCount = 0
        for key in topic_list:
          path = "/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/Training/"+key+".xml"
          trainDataSize = trainDataSize
          #validationDataSize = validationDataSize
          #print(trainDataSize)
          with open(path,'r',encoding='utf-8') as file:
            content = file.read()
            soup = bs(content)
            rowList = soup.findAll("row")
            #correctCount = 0
            #print(trainDataSize)
            for row in rowList[trainDataSize+validationDataSize+loop*testSize:trainDataSize++validationDataSize+loop*testSize+testSize]:
              #print(row.get('body'))
              #print()
              if len(row.get('body')) == 0:
                continue
              word_list = preprocessText(row.get('body'))
              if len(word_list) == 0:
                continue
              if predictNaive(word_list,laplaceSmoothingCalc,topicLength,topicWiseCount,uniqueWordSize,smFactor) is topic_list[key]:
                correctCount += 1
              totalCount += 1

        acc.append(correctCount/float(totalCount))
    return acc 

def runKNN(trainDataStore,validdataStore,topic_list,topicLength):

  def BinaryRepresent(trainDataStore,position_array,topic_list,testStore):
    #find unique word
    uniqueWord = []
    for pair in trainDataStore:
      wordList = trainDataStore[pair]
      for word in wordList:
        if word not in uniqueWord:
          uniqueWord.append(word)
    
    for pair in testStore:
      wordList = testStore[pair]
      for word in wordList:
        if word not in uniqueWord:
          uniqueWord.append(word)
    #print(uniqueWord)

    # binary representation creation
    binary_represent_final_train = {}
    binary_represent_final_test = {}
    bow_represent_final_train = {}
    bow_represent_final_test = {}
    
    for pair in trainDataStore:
      wordList = list(trainDataStore[pair])
      tempStore = []
      tempStoreBow = []
      for word in uniqueWord:
        if word in wordList:
          tempStore.append(1)
        else:
          tempStore.append(0)
        tempStoreBow.append(wordList.count(word))
      binary_represent_final_train[pair] = tempStore
      bow_represent_final_train[pair] = tempStoreBow

    for pair in testStore:
      wordList = list(testStore[pair])
      tempStore = []
      tempStoreBow = []
      for word in uniqueWord:
        if word in wordList:
          tempStore.append(1)
        else:
          tempStore.append(0)
        tempStoreBow.append(wordList.count(word))
      binary_represent_final_test[pair] = tempStore  
      bow_represent_final_test[pair] = tempStoreBow
    
    return binary_represent_final_train,uniqueWord,binary_represent_final_test,bow_represent_final_train,bow_represent_final_test

  def hammingDistance(trainArray,testArray):
    dist = list(distance.cdist(trainArray,[testArray],'cityblock').flatten())
    return dist

  def predictionBinary(header,testingBinArr,binary_represent_final_test,dataName):
    dist = list(hammingDistance(testingBinArr,binary_represent_final_test[dataName]))
    dist, header = (list(t) for t in zip(*sorted(zip(dist, header))))
    return header

  def euclideanDistance(trainArray,testArray):
    distance = np.sqrt(np.sum(np.square(np.subtract(trainArray,testArray)),axis=1))
    return distance 

  def predictionBow(headerBow,testingBinArrBow,bow_represent_final_test,dataName):
    distance = list(euclideanDistance(testingBinArrBow,bow_represent_final_test[dataName]))
    distance, headerBow = (list(t) for t in zip(*sorted(zip(distance, headerBow))))
    return headerBow     

  def performanceEvaluationKNN(trainDataStore,validdataStore,position_array,topic_list):
    
    binary_represent_final_train,uniqueWord,binary_represent_final_test,bow_represent_final_train,bow_represent_final_test = BinaryRepresent(trainDataStore,position_array,topic_list,validdataStore)
    #print(bow_represent_final_train)
    print("bow represent created")
    ####################
    header = list(binary_represent_final_train.keys())
    testingBinArr = np.array(list(binary_represent_final_train.values()))
  
    headerBow = list(bow_represent_final_train.keys())
    testingBinArrBow = np.array(list(bow_represent_final_train.values()))
      
    # binary representation
    print("bin training started")
    allBinDisctance = {}
    for dataName in binary_represent_final_test:
        predicted = predictionBinary(header,testingBinArr,binary_represent_final_test,dataName)
        allBinDisctance[dataName] = predicted

    n_neighbors = [1,3,5]
    for neighbor in n_neighbors:
      print("Neighbor :",neighbor)
      totalCount = 0
      correctCount = 0
      for dataName in binary_represent_final_test:
        predArray = allBinDisctance[dataName]
        voting_count = {}
        for val in predArray[:neighbor]:
          if val[0] in voting_count:
            voting_count[val[0]] += 1
          else :
            voting_count[val[0]] = 1

        prediction = max(voting_count, key=voting_count.get)
        if prediction == dataName[0]:
          correctCount += 1
        totalCount += 1
        
      print("Total Correct Count: ",correctCount," Total Wrong Count: ",totalCount-correctCount," Accuracy: ",(correctCount*100)/(totalCount))
    print()
    
    # bag of words
    print("bow training started")
    allBinDisctance = {}
    for dataName in bow_represent_final_test:
        predicted = predictionBow(headerBow,testingBinArrBow,bow_represent_final_test,dataName)
        allBinDisctance[dataName] = predicted

    n_neighbors = [1,3,5]
    for neighbor in n_neighbors:
      print("Neighbor :",neighbor)
      totalCount = 0
      correctCount = 0
      for dataName in bow_represent_final_test:
        predArray = allBinDisctance[dataName]
        voting_count = {}
        for val in predArray[:neighbor]:
          if val[0] in voting_count:
            voting_count[val[0]] += 1
          else :
            voting_count[val[0]] = 1

        prediction = max(voting_count, key=voting_count.get)
        if prediction == dataName[0]:
          correctCount += 1
        totalCount += 1
        
      print("Total Correct Count: ",correctCount," Total Wrong Count: ",totalCount-correctCount," Accuracy: ",(correctCount*100)/(totalCount))
    print()
    
    
  performanceEvaluationKNN(trainDataStore,validdataStore,position_array,topic_list)



def runTFIDF(datastore,validstore,topic_list,topicLength,type):

  def numberDocsForWord(datastore,words):
    wordVal = {}
    value = list(datastore.values())
    for word in words:
      count = sum(word in e for e in value)    
      wordVal[word] = count
    return wordVal

  def uniqueWordFunc(trainDataStore,testStore):
    uniqueWord = []
    for pair in trainDataStore:
      wordList = trainDataStore[pair]
      for word in wordList:
        if word not in uniqueWord:
          uniqueWord.append(word)
    wordval = numberDocsForWord(datastore,uniqueWord)
    for pair in testStore:
      wordList = testStore[pair]
      for word in wordList:
        if word not in uniqueWord:
          uniqueWord.append(word)
    return uniqueWord,wordval

  def countData(datastore,validstore,uniqueWord):
    countDatastore = {}
    countDatastoreTest = {}
    
    for key in datastore:
      tempArray = list(datastore[key])
      wordDictA = []
      for word in uniqueWord:
        wordDictA.append(tempArray.count(word)/float(len(tempArray)))
      countDatastore[key] = wordDictA
      
    for key in validstore:
      tempArray = list(validstore[key])
      wordDictA = []
      for word in uniqueWord:
        wordDictA.append(tempArray.count(word)/float(len(tempArray)))
      countDatastoreTest[key] = wordDictA
    
    return countDatastore,countDatastoreTest

  uniqueWord,wordval = uniqueWordFunc(datastore,validstore)    
  tfData,tfDataTest = countData(datastore,validstore,uniqueWord)
  print(len(uniqueWord))

  def computeIDF(datastore,validstore,numberofDoc,dictionaryWords,wordval,tfData,tfDataTest):
    
    idfDict = {}
    idfDict = dict.fromkeys(set(dictionaryWords), 0)
    
    for key in datastore:
      tempArray = datastore[key]
      temp = []
      for word in dictionaryWords:
        if word in tempArray:
          temp.append(math.log10(numberofDoc / (float(wordval[word]) + 1)))
        else :
          temp.append(0.0)
      tfData[key] = list(np.multiply(temp,tfData[key]))

    for key in validstore:
      tempArray = validstore[key]
      temp = []
      for word in dictionaryWords:
        if word in tempArray and word in wordval:
          temp.append(math.log10(numberofDoc / (float(wordval[word]) + 1)))
        else :
          temp.append(0.0)
      tfDataTest[key] = list(np.multiply(temp,tfDataTest[key]))
        
    return tfData,tfDataTest

  numberofDoc = len(datastore)
  #print(numberofDoc)

  idfData = {}
  tfidfDataArray,tfidfDataTestArray = computeIDF(datastore,validstore,numberofDoc,uniqueWord,wordval,tfData,tfDataTest)
  
  def cosineSimilarity(headerCS,testingBinArrCS,tfidfDataTestArray,keyval):
    listVal1 = np.sqrt(np.sum(np.square(testingBinArrCS),axis=1))
    listVal2 = np.sqrt(np.sum(np.square(tfidfDataTestArray[keyval])))
    dotResult = np.dot(testingBinArrCS,tfidfDataTestArray[keyval])
    finalResult = np.divide(dotResult,np.dot(listVal1,listVal2))
    finalResult, headerCS = (list(t) for t in zip(*sorted(zip(finalResult, headerCS),reverse=True)))
    return headerCS

  headerCS = list(tfidfDataArray.keys())
  testingBinArrCS = np.array(list(tfidfDataArray.values()))

  allBinDisctance = {}
  for keyval in tfidfDataTestArray:
    predicted = cosineSimilarity(headerCS,testingBinArrCS,tfidfDataTestArray,keyval)
    allBinDisctance[keyval] = predicted

  if type is 'val':
    n_neighbors = [1,3,5]
  elif type is 'test':
    n_neighbors = [5]

  for neighbor in n_neighbors:
    print("Neighbor :",neighbor)
    totalCount = 0
    correctCount = 0
    for keyval in tfidfDataTestArray:
      predArray = allBinDisctance[keyval]
      voting_count = {}
      for val in predArray[:neighbor]:
        if val[0] in voting_count:
          voting_count[val[0]] += 1
        else :
          voting_count[val[0]] = 1

      prediction = max(voting_count, key=voting_count.get)
      if prediction == keyval[0]:
        correctCount += 1
      totalCount += 1
      
    print("Total Correct Count: ",correctCount," Total Wrong Count: ",totalCount-correctCount," Accuracy: ",(correctCount*100)/(totalCount))
  print()
  return correctCount/float(totalCount)

train_path = "/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/Training/"
#topic name reading
topic_number =0
topic_list = {}
with open("/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/topics.txt") as fp:
  lines = fp.readlines()
   
  for line in lines:
    line = line.strip()
    topic_list[line] = topic_number
    topic_number += 1
    
    if topic_number == 3 :
      break
    
fp.close()
print(topic_list)
topicLength = len(topic_list)
# train data and validation data 
topicDataSize = 700
trainDataSize = 500
validationDataSize = 200
allResult = {}
trainDataStore = {}
validdataStore = {}
count = 0

position_array = []
for i in range(0,topicDataSize):
  position_array.append(i)

for topic in topic_list:
  
  with open(train_path+topic+".xml",'r',encoding='utf-8') as file:
    content = file.read()
    soup = bs(content)
    rowList = soup.findAll("row")
    count = 0
    for row in rowList[0:trainDataSize]:
      topicPair = (topic,position_array[count])
      if len(row.get('body')) == 0:
        continue;
      word_list = preprocessText(row.get('body'))
      if len(word_list) == 0:
        continue
      for word in word_list:
        pairWord = (word,topic_list[topic])
        if pairWord in allResult:
          allResult[pairWord] += 1
        else:
          allResult[pairWord] = 1
      count += 1
      trainDataStore[topicPair] = np.array(word_list)

    #validation data
    for i in range(trainDataSize,trainDataSize+validationDataSize):
      topicPair = (topic,position_array[i])
      row = rowList[position_array[i]]
      if len(row.get('body')) == 0:
        continue
      word_list = preprocessText(row.get('body'))
      if len(word_list) == 0:
        continue
      validdataStore[topicPair] = np.array(word_list)

print("Document count :",count)

print("Naive bayes running...")
smfactorList = [.001,.005,.01,.05,.08,0.1,0.3,0.5,0.8,1,10,20]
accuracy = naiveBayes(allResult,trainDataSize,validationDataSize,topic_list,topicLength,smfactorList,'val')
print("KNN running..")
runKNN(trainDataStore,validdataStore,topic_list,topicLength)
print("TF-IDF running...")
runTFIDF(trainDataStore,validdataStore,topic_list,topicLength,'val')

#######################################
# Test Naive Bayes
smfactorList = [.05]
accuracyList = naiveBayes(allResult,trainDataSize,validationDataSize,topic_list,topicLength,smfactorList,'test')
print(accuracyList)
print(len(accuracyList))

###########################################
# Test TFIDF
totalLoop = 3
acc = []
for loop in range(totalLoop):
  testSize = 10
  position_array = []
  for i in range(topicDataSize+loop*testSize,topicDataSize+loop*testSize+testSize):
    position_array.append(i)
  testDatastore = {}

  for topic in topic_list:
    with open(train_path+topic+".xml",'r',encoding='utf-8') as file:
      content = file.read()
      soup = bs(content)
      rowList = soup.findAll("row")
      
      for i in range(len(position_array)):
        topicPair = (topic,position_array[i])
        row = rowList[position_array[i]]
        if len(row.get('body')) == 0:
          continue
        word_list = preprocessText(row.get('body'))
        if len(word_list) == 0:
          continue
        testDatastore[topicPair] = np.array(word_list) 

  accuracy = runTFIDF(trainDataStore,testDatastore,topic_list,topicLength,'test')
  acc.append(accuracy)

print(acc)
print(len(acc))

tStat,p = scipy.stats.ttest_rel(accuracyList,acc)
print("t-stat value:",tStat)
print("p value:",p)

sig_level = [0.005,.01,.05]

for sig in sig_level:
  print("Significance level :",sig)
  if p>sig:
    print("Null hypothesis occurs")
    print("Similar mean and similar method")
    print()
  else:
    print("Means are not similar and different method")
    print()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
{'Coffee': 0, 'Arduino': 1, 'Anime': 2}
Document count : 485
Naive bayes running...
smoothing factor constant : 0.001
98.11643835616438

smoothing factor constant : 0.005
98.11643835616438

smoothing factor constant : 0.01
98.28767123287672

smoothing factor constant : 0.05
98.11643835616438

smoothing factor constant : 0.08
98.11643835616438

smoothing factor constant : 0.1
98.11643835616438

smoothing factor constant : 0.3
97.77397260273972

smoothing factor constant : 0.5
97.77397260273972

smoothing factor constant : 0.8
97.94520547945206

smoothing factor constant : 1
97.94520547945206

smoothing factor constant : 10
96.57534246575342

smoothing factor constant :